In [9]:
#s3_prediction = "s3://aegovan-data/chemprot/predictions_chemprot-bert-f1-2022-05-22-23-18-21-560/prediction_chemprot_test.json"
s3_test=[
"s3://aegovan-data/counterfactual_yelp/predictions_counterfact-imdb-simple-2023-04-03-01-35-49-441/2023040817/yelp_test.csv.json" # .25
, "s3://aegovan-data/counterfactual_amazon/predictions_counterfact-imdb-simple-2023-04-03-01-35-49-441/2023040817/amazon_test.csv.json" # .25
, "s3://aegovan-data/counterfactual_semeval/predictions_counterfact-imdb-simple-2023-04-03-01-35-49-441/2023040913/semeval_SemEval2017-task4-test.subtask-BD.english.txt.json"  # .25
    
, "s3://aegovan-data/counterfactual_yelp/predictions_counterfact-imdb-simple-2023-04-09-02-17-57-367/2023040911/yelp_test.csv.json" # .1,
, "s3://aegovan-data/counterfactual_semeval/predictions_counterfact-imdb-simple-2023-04-09-02-17-57-367/2023040912/semeval_SemEval2017-task4-test.subtask-BD.english.txt.json" #.1
, "s3://aegovan-data/counterfactual_amazon/predictions_counterfact-imdb-simple-2023-04-09-02-17-57-367/2023040911/amazon_test.csv.json" #.1
    
, "s3://aegovan-data/counterfactual_semeval/predictions_counterfact-imdb-original-2023-04-09-19-03-45-522/2023040913/semeval_SemEval2017-task4-test.subtask-BD.english.txt.json" # 0.9 original
, "s3://aegovan-data/counterfactual_yelp/predictions_counterfact-imdb-original-2023-04-09-19-03-45-522/2023040913/yelp_test.csv.json"    # 0.9 original
, "s3://aegovan-data/counterfactual_amazon/predictions_counterfact-imdb-original-2023-04-09-19-03-45-522/2023040913/amazon_test.csv.json"
]

train_key_adv_map = {
    "predictions_counterfact-imdb-simple-2023-04-03-01-35-49-441" :.25
    ,"predictions_counterfact-imdb-simple-2023-04-09-02-17-57-367" : .1
   ,  "predictions_counterfact-imdb-original-2023-04-09-19-03-45-522" : 0.9

}

In [2]:
import sys, os, pandas as pd
sys.path.append("../src")

import utils

In [3]:
from utils.s3_utils import download_file as download_s3_file


In [4]:
local_temp ="./temp"

In [5]:
local_prediction_dir = os.path.join(local_temp, "prediction")
os.makedirs(local_prediction_dir, exist_ok=True)

def load(s3_test):
    result = []
    for uri in s3_test:
        local_test_file  = download_s3_file( uri, local_prediction_dir)
        df_test = pd.read_json(local_test_file)
        df_test["uri"] = uri
        result.append(df_test)
    
    return result

result = load(s3_test)

In [6]:
result[0].head()

,Sentiment,Text,Negative,Positive,prediction,confidence,uri
0,Positive,"Contrary to other reviews, I have zero complai...",0.020631,0.979369,Positive,0.979369,s3://aegovan-data/counterfactual_yelp/predicti...
1,Negative,Last summer I had an appointment to get new ti...,0.276431,0.723569,Positive,0.723569,s3://aegovan-data/counterfactual_yelp/predicti...
2,Positive,"Friendly staff, same starbucks fair you get an...",0.060069,0.939931,Positive,0.939931,s3://aegovan-data/counterfactual_yelp/predicti...
3,Negative,The food is good. Unfortunately the service is...,0.992037,0.007963,Negative,0.992037,s3://aegovan-data/counterfactual_yelp/predicti...
4,Positive,Even when we didn't have a car Filene's Baseme...,0.014551,0.985449,Positive,0.985449,s3://aegovan-data/counterfactual_yelp/predicti...


In [17]:
from sklearn.metrics import accuracy_score
def consolidate_result(df_list): 
    result = []
    for df in df_list:

        score = accuracy_score(df["Sentiment"], df["prediction"])
        result.append({
            "Train" : "IMDB-C",
            "TrainSize" : "3.4K",
            "Test" : df["uri"].iloc[0].split("/")[3],
            "TestSize": len(df),
            "h_adv_p_n" : train_key_adv_map[ df["uri"].iloc[0].split("/")[4]],
            "Score" : round(score*100, 1),
            "Uri" : df["uri"].iloc[0]


        })
    return pd.DataFrame(result).sort_values(by=["Test", "h_adv_p_n"])

df_scores = consolidate_result(result)

In [20]:
print(df_scores[[ "Train","TrainSize", "Test", "TestSize", "h_adv_p_n", "Score"]].to_latex(index = False))

\begin{tabular}{lllrrr}
\toprule
Train & TrainSize &                    Test &  TestSize &  h\_adv\_p\_n &  Score \\
\midrule
 IMDB &      3.4K &   counterfactual\_amazon &    400000 &       0.10 &   87.1 \\
 IMDB &      3.4K &   counterfactual\_amazon &    400000 &       0.25 &   89.9 \\
 IMDB &      3.4K &   counterfactual\_amazon &    400000 &       0.90 &   88.6 \\
 IMDB &      3.4K &  counterfactual\_semeval &      6069 &       0.10 &   62.7 \\
 IMDB &      3.4K &  counterfactual\_semeval &      6069 &       0.25 &   78.3 \\
 IMDB &      3.4K &  counterfactual\_semeval &      6069 &       0.90 &   70.2 \\
 IMDB &      3.4K &     counterfactual\_yelp &     37986 &       0.10 &   88.3 \\
 IMDB &      3.4K &     counterfactual\_yelp &     37986 &       0.25 &   90.2 \\
 IMDB &      3.4K &     counterfactual\_yelp &     37986 &       0.90 &   89.4 \\
\bottomrule
\end{tabular}

